# 滑动平均模型可以使模型在测试数据上更加健壮。
在tensorflow中提供了tf.train.ExponentialMovingAverage来实现滑动平均模型
在初始化ExponentialMovingAverage时，需要提供一个衰减率，这个衰减率将用于控制模型更新的速度。
ExponentialMovingAverage会给每个变量维护一个影子变量，影子变量初始值就是相应变量的初始值，而每次运行变量更新时，影子变量的值会更新为:
    shadow_variable = decay*shadow_variable+(1-decay)*variable
其中shadow_variable为影子变量，variable为待更新变量，decay为衰减率。decay决定了模型更新的速度，decay越大模型越趋于稳定。在实际应用中，
decay会呗设成非常接近1的数字（0.999或0.9999），为了使得模型在训练前期可以更新得更快，ExponentialMovingAverage还提供了num_updates参数
来动态设置decay的大小。如果在ExponentialMovingAverage初始化时提供了num_updates参数，那么每次使用的衰减率将时
min{decay,(1+num_updates)/(10+num_updates)}

# 模拟滑动平均的过程

In [1]:
import tensorflow as tf

In [4]:
#定义一个变量用于滑动平均，这个变量初始值为0。注意这里手动指定了变量的类型为tf.float32,因为所有需要计算滑动平均的变量必须是实数型
v1 = tf.Variable(0,dtype=tf.float32)
#定义神经网络的当前模拟步数，此处trainable设为False时则该变量不会加入collection中
step = tf.Variable(0,trainable=False)

ema = tf.train.ExponentialMovingAverage(0.99,step)
#apply求得移动平均的值
maintain_averages_op = ema.apply([v1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    #average会返回对应列表的移动平均值
    print(sess.run([v1,ema.average(v1)]))
    #更新v1的值为5
    sess.run(tf.assign(v1,5))
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    
    #更新v1的值为10
    sess.run(tf.assign(v1,10))
    sess.run(tf.assign(step,100000))
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.5549998]
